The returned data structure should include:

* the total price for the order
* the total VAT for the order
* price and VAT for each item in the order

Load example order and pricing dicts

In [1]:
# provided
pricing = {
	"prices": [
		{
			"product_id": 1,
			"price": 599,
			"vat_band": "standard"
		},
		{
			"product_id": 2,
			"price": 250,
			"vat_band": "zero"
		},
		{
			"product_id": 3,
			"price": 250,
			"vat_band": "zero"
		},
		{
			"product_id": 4,
			"price": 1000,
			"vat_band": "zero"
		},
		{
			"product_id": 5,
			"price": 1250,
			"vat_band": "standard"
		}
	],
	"vat_bands": {
		"standard": 0.2,
		"zero": 0
	}
}

In [2]:
# format into a dict where key is product_id
pricing_formatted = {product["product_id"]:
                                             {
                                              "price":product["price"], 
                                              "vat_rate":pricing["vat_bands"][product["vat_band"]],
                                              "VAT":round(pricing["vat_bands"][product["vat_band"]]*product["price"], 2) # round to nearest penny
                                             } 
                     for product in pricing["prices"]}

In [3]:
pricing_formatted

{1: {'price': 599, 'vat_rate': 0.2, 'VAT': 119.8},
 2: {'price': 250, 'vat_rate': 0, 'VAT': 0},
 3: {'price': 250, 'vat_rate': 0, 'VAT': 0},
 4: {'price': 1000, 'vat_rate': 0, 'VAT': 0},
 5: {'price': 1250, 'vat_rate': 0.2, 'VAT': 250.0}}

In [4]:
# Provided. assume order is from a single customer
order_request = {
    "order": {
        "id": 12345,
        "customer": {
            ...
        },
        "items": [
            {
                "product_id": 1,
                "quantity": 1
            },
            {
                "product_id": 2,
                "quantity": 5
            },
            {
                "product_id": 3,
                "quantity": 1
            }
        ]
    }
}

In [5]:
# Get a dict where the key is the product_id and value is the quantity
orders = {item["product_id"]:{"quantity":item["quantity"]} for item in order_request["order"]["items"]}
orders

{1: {'quantity': 1}, 2: {'quantity': 5}, 3: {'quantity': 1}}

Join the two dicts on product_id (key)

In [6]:
_ = [orders[k].update(pricing_formatted[k]) for k in orders] # one liner to update orders

In [7]:
orders

{1: {'quantity': 1, 'price': 599, 'vat_rate': 0.2, 'VAT': 119.8},
 2: {'quantity': 5, 'price': 250, 'vat_rate': 0, 'VAT': 0},
 3: {'quantity': 1, 'price': 250, 'vat_rate': 0, 'VAT': 0}}

In [8]:
[order["VAT"] for order in orders.values()]

[119.8, 0, 0]

We can now process the orders

In [9]:
def calc_order_details(orders):
    order_details = {}
    order_details["orders"] = orders
    
    order_details["total_price"] = sum([order["price"]*order["quantity"] for order in orders.values()])
    order_details["total_VAT"] = sum([order["VAT"]*order["quantity"] for order in orders.values()])
    return order_details

In [10]:
calc_order_details(orders)

{'orders': {1: {'quantity': 1, 'price': 599, 'vat_rate': 0.2, 'VAT': 119.8},
  2: {'quantity': 5, 'price': 250, 'vat_rate': 0, 'VAT': 0},
  3: {'quantity': 1, 'price': 250, 'vat_rate': 0, 'VAT': 0}},
 'total_price': 2099,
 'total_VAT': 119.8}